In [4]:
!python --version
!pip list | grep tensorflow

Python 3.6.12 :: Anaconda, Inc.
tensorflow           1.15.4
tensorflow-estimator 1.15.1


In [6]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, SimpleRNN, LSTM, Input, GRU
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist



In [8]:
!python --version
!pip list | grep tensorflow

Python 3.6.12 :: Anaconda, Inc.
tensorflow           1.15.4
tensorflow-estimator 1.15.1


## Clean Data

In [5]:
# load mnist dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# compute the number of labels
num_labels = len(np.unique(y_train))

# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# resize and normalize
#image_size = x_train.shape[1]
#print(x_train.shape)

x_train = np.reshape(x_train,[x_train.shape[0], -1, 1])
x_test = np.reshape(x_test,[x_test.shape[0], -1, 1])

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print("x_train", x_train.shape)
print("y_train", y_train.shape)

print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train (60000, 784, 1)
y_train (60000, 10)
x_test (10000, 784, 1)
y_test (10000, 10)


## Setup Model

In [7]:
# network parameters
input_shape = (x_train.shape[1:])
batch_size = 128
units = 100
dropout = 0.00

# model is RNN with 256 units, input is 28-dim vector 28 timesteps
model = Sequential()
model.add(Input(input_shape, name="input"))
model.add(GRU(units=units,
                    dropout=dropout, name="GRU" ))
                    #input_shape=input_shape))
model.add(Dense(num_labels, name="dense"))
model.add(Activation('softmax', name="activate"))
model.summary()
print("units", units)
print("input_shape", input_shape)
print("num_labels", num_labels)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
GRU (GRU)                    (None, 100)               30600     
_________________________________________________________________
dense (Dense)                (None, 10)                1010      
_________________________________________________________________
activate (Activation)        (None, 10)                0         
Total params: 31,610
Trainable params: 31,610
Non-trainable params: 0
_________________________________________________________________
units 100
input_shape (784, 1)
num_labels 10


## Model compile and fit (train)

In [ ]:
# enable this if pydot can be installed
# pip install pydot
#plot_model(model, to_file='rnn-mnist.png', show_shapes=True)

# loss function for one-hot vector
# use of sgd optimizer
# accuracy is good metric for classification tasks

from tensorflow.keras.optimizers import RMSprop

learning_rate = 1e-3
rmsprop = RMSprop(lr=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer= rmsprop,
              metrics=['accuracy'])

#model.compile(loss='categorical_crossentropy',
#              optimizer='sgd',
#              metrics=['accuracy'])
# train the network

model.fit(x_train, y_train, 
          epochs=200, 
          batch_size=batch_size,
          validation_data=(x_test, y_test))



Epoch 1/200
469/469 [==============================] - 22s 36ms/step - loss: 2.3025 - accuracy: 0.1201 - val_loss: 2.0546 - val_accuracy: 0.2155
Epoch 2/200
469/469 [==============================] - 17s 35ms/step - loss: 1.9257 - accuracy: 0.2558 - val_loss: 1.7856 - val_accuracy: 0.3371
Epoch 3/200
469/469 [==============================] - 16s 35ms/step - loss: 1.7281 - accuracy: 0.3515 - val_loss: 1.7072 - val_accuracy: 0.3522
Epoch 4/200
469/469 [==============================] - 16s 35ms/step - loss: 1.7018 - accuracy: 0.3678 - val_loss: 1.6968 - val_accuracy: 0.3774
Epoch 5/200
469/469 [==============================] - 16s 35ms/step - loss: 1.7039 - accuracy: 0.3579 - val_loss: 1.6647 - val_accuracy: 0.4054
Epoch 6/200
469/469 [==============================] - 16s 35ms/step - loss: 1.6753 - accuracy: 0.3907 - val_loss: 1.6775 - val_accuracy: 0.3772
Epoch 7/200
469/469 [==============================] - 16s 35ms/step - loss: 1.6581 - accuracy: 0.3891 - val_loss: 1.8016 - val_ac

## Model pred

In [ ]:
_, acc = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))